In [1]:
import numpy as np
import pandas as pd
import wandb
from wd.utils.utilities import mlflow_linearize

In [ ]:
from IPython.core.display_functions import display


def get_runs(run_filters):
    # Project is specified by <entity/project-name>
    runs = api.runs("pasqualedem/weeddetection", filters=run_filters)

    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k,v in run.config.items()
              if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)
    df = pd.DataFrame([{**mlflow_linearize(dc), **mlflow_linearize(ds)} for dc, ds in zip(config_list, summary_list)])
    return df

def remove_hyp(df):
    to_delete = filter(lambda x: x.startswith('hyper_params'), df.columns)
    display(list(to_delete))
    for l in to_delete:
        df.drop(l, inplace=True)

def rename_values(df: pd.DataFrame, vrenamings: dict):
    for column, values in vrenamings.items():
        if column in df.columns:
            df[column] = df[column].replace(values)

In [ ]:
api = wandb.Api()

In [ ]:
renamings = {
    'in_params.dataset.channels': 'Channels',
    'in_params.train_params.optimizer_params.weight_decay': "Weight Decay",
    'in_params.train_params.initial_lr': 'LR',
    'in_params.model.name': 'Model',
    'in_params.model.params.backbone': 'Backbone',
    'in_params.train_params.freeze_pretrained': 'Strategy',
    'in_params.model.params.main_pretrained': 'Used Weights',
    'train_WrapF1': 'Train F1',
    'f1': 'F1',
    'in_params.dataset.batch_size': 'Batch Size',
    'precision': 'Precision',
    'in_params.train_params.loss.params.gamma': 'Gamma',
    'in_params.model.params.fusion_type': 'DropPath',
    'recall': 'Recall'
}
value_renamings = {
    'Strategy': {True: 'Freeze', False: 'Fine-Tuning'},
    'DropPath': {'conv_sum_drop': 'Yes', 'conv_sum': 'No'}
}
channels_ordering = {
    "G, NDVI, NIR, R, RE" : 1,
    "R, G, NIR, RE": 2,
    "R, G, RE, NIR": 2,
    "R, G, RE": 3,
    "G, NIR, R": 3,
    "R, G, NIR": 4,
    "R, G, NDVI": 5,
    "R, G": 6,
    "CIR": 7,
    "NDVI": 8,

    'Fine-Tuning': 1,
    'Freeze': 2,

    'R': 1,
    'G': 2,
    'RGB': 2.5,
    'G, G, R': 3,
    'R, G, G, G': 4,

    'MiT-B0': 0,
    'MiT-B1': 1,
    'lawin': 0,
    'laweed': 1,
    'doublelawin': 2,
    'doublelaweed': 3,
}

## Lawin

In [ ]:
filters = {
    'group': 'Lawin', 'config.in_params.model.params.backbone': 'MiT-B0'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    # 'in_params.model.name',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]

In [ ]:
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by='Channels')['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by="Channels", key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

In [ ]:
selected_runs

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/lawin.csv")

In [ ]:
selected_runs.to_csv("../../exps/lawin-complete.csv")

## Lawin Aug

In [ ]:
filters = {
    'group': 'LawinAugPatience'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by='Channels')['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by="Channels", key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

In [ ]:
best_runs.to_csv("../../exps/lawin_aug.csv")

In [ ]:
selected_runs.to_csv("../../exps/lawin_aug-complete.csv")

## Lawin Pretrained

In [ ]:
filters = {
    '$or': [{'group': 'LawinPretrained'}, {'group': 'LawinFrozen'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/lawin_pretrained.csv")
selected_runs.to_csv("../../exps/lawin_pretrained-complete.csv")

## Focal

In [ ]:
filters = {
    'group': 'Focal'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/focal.csv")
selected_runs.to_csv("../../exps/focal-complete.csv")

## SRR

In [ ]:
filters = {
    'group': 'RandomRotation'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs
selected_runs.fillna("", inplace=True)

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/rotation.csv")
selected_runs.to_csv("../../exps/rotation-complete.csv")

## Double Encoder

In [ ]:
filters = {
    'group': 'DoubleLawin'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs
selected_runs.fillna("", inplace=True)

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/doublelawin.csv")
selected_runs.to_csv("../../exps/doublelawin-complete.csv")

## Double Encoder Pre-Trained

In [ ]:
filters = {
    'group': 'DoubleLawinPretrained'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'train_WrapF1',
    'f1',
    'precision',
    'recall'
]
roundings = ['Train F1', 'F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.fillna(0, inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs['Gamma'] = selected_runs['Gamma'].astype(float).astype(int)

In [ ]:
selected_runs

In [ ]:
idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
selected_runs.mean()

In [ ]:
selected_runs.groupby(by=['Gamma']).mean()

In [ ]:
selected_runs.to_csv("../../exps/doublelawinpretrained-complete.csv")
best_runs.to_csv("../../exps/doublelawinpretrained.csv")

## Double Encoder Pre-Trained Drop

In [ ]:
filters = {
    'group': 'DLPretrainedDrop'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.params.backbone',
    'in_params.train_params.loss.params.gamma',
    'train_WrapF1',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall', 'Train F1']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.fillna("", inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
selected_runs

In [ ]:
idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
selected_runs.groupby(by=['Gamma']).mean()

In [ ]:
selected_runs.mean()

In [ ]:
selected_runs.to_csv("../../exps/dlpretraineddrop-complete.csv")
best_runs.to_csv("../../exps/dlpretraineddrop.csv")

## RGB Weights

In [ ]:
filters = {
    'group': 'RedEdge'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.main_pretrained',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.fillna(0, inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
selected_runs['Gamma'] = selected_runs['Gamma'].astype(float).astype(int)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Backbone', 'Channels', 'Strategy', 'Used Weights'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

In [ ]:
selected_runs

In [ ]:
selected_runs.to_csv("../../exps/rgb_weights-complete.csv")

In [ ]:
filters = {
    '$or': [{'group': 'RedEdge'}, {'group': 'RandomRotation'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'f1',
]
roundings = ['F1']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs.loc[(selected_runs['Channels'] == 'CIR') & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'RGB'
selected_runs.loc[(selected_runs['Channels'].isin([['NDVI']])) & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'R'
selected_runs.drop(selected_runs[selected_runs['Channels'].isin([['R', 'G']])].index, inplace=True)

In [ ]:
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Backbone', 'Channels', 'Used Weights'],
                          key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Strategy', 'Used Weights'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels', 'Strategy'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
compare_runs = best_runs.merge(best_runs, on=['Channels', 'Strategy'])
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == compare_runs['Used Weights_y']].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == 'G, G, R'].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == 'G'].index, inplace=True)
compare_runs

In [ ]:
complete = best_runs[best_runs['Channels'] == 'R, G, NIR, RE']
complete.rename(columns={'Gamma': 'Gamma_y', 'F1': 'F1_y', 'Used Weights': 'Used Weights_y', 'Backbone': 'Backbone_x'}, inplace=True)
compare_runs = compare_runs.append(complete)
compare_runs.fillna("", inplace=True)
compare_runs

In [ ]:
compare_runs.to_csv("../../exps/rgb_weights.csv")

## Laweed

In [ ]:
filters = {
    'group': 'Laweed'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
selected_runs

In [ ]:
best_runs

In [ ]:
selected_runs.to_csv("../../exps/laweed-complete.csv")

In [ ]:
filters = {
    '$and': [
        {'$or': [{'group': 'Laweed'}, {'group': 'RandomRotation'}, {'group': 'RedEdge'}]},
        {'config.in_params.model.params.backbone': 'MiT-B0'}
    ]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'in_params.model.name',
    'f1',
]
roundings = ['F1']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs
selected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['Gamma'].fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Channels'],
                          key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
selected_runs.loc[(selected_runs['Channels'] == 'CIR') & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'RGB'
selected_runs.loc[(selected_runs['Channels'].isin([['NDVI']])) & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'R'

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Model'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Model', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
best_runs

In [ ]:
compare_runs = best_runs.merge(best_runs, on=['Channels'])
compare_runs.drop(compare_runs[compare_runs['Model_x'] == compare_runs['Model_y']].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Model_x'] == 'laweed'].index, inplace=True)
compare_runs

In [ ]:
compare_runs.drop(columns=['Model_x', 'Model_y'], inplace=True)
compare_runs

In [ ]:
compare_runs.to_csv("../../exps/laweed.csv")

## Double Laweed

In [ ]:
filters = {
    '$or': [{'group': 'DoubleLaweedPrBatch'}, {'group': 'DoubleLaweedPrDropBatch'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.fusion_type',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs['DropPath'].fillna('conv_sum', inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['Gamma'].fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
selected_runs

In [ ]:
selected_runs.to_csv("../../exps/doublelaweed-complete.csv")

In [ ]:
filters = {
    '$or': [{'group': 'DoubleLaweedPrBatch'}, {'group': 'DoubleLaweedPrDropBatch'},{'group': 'DLPretrainedDrop'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.fusion_type',
    'in_params.model.name',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [ ]:
selected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs['DropPath'].fillna('conv_sum', inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['Gamma'].fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Channels'],
                          key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)



In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Model'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Model', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
best_runs

In [ ]:
compare_runs = best_runs.merge(best_runs, on=['Channels'])

In [ ]:
compare_runs

In [ ]:
compare_runs.drop(compare_runs[compare_runs['Model_x'] == compare_runs['Model_y']].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Model_x'] == 'doublelaweed'].index, inplace=True)
compare_runs

In [ ]:
compare_runs.to_csv("../../exps/doublelaweed.csv")

# Final results

In [ ]:
filters = {
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Model', 'Backbone'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
filters = {
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.name',
    'in_params.model.params.backbone',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters)
remove_hyp(runs_df)

projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs_channel = selected_runs[idx]
best_runs_channel[roundings] = best_runs_channel[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)
best_runs_channel